In [80]:
import cv2
import numpy as np
import time


In [81]:
classNames= []
classFilePath = 'data/coco.names'
with open(classFilePath,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

In [82]:
configPath = 'data/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'data/frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 000001CF1E90EF10>

In [160]:
rectColor =(244,5,20)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
fontSize = 1
fontColor = (255, 0, 255)

thres = 0.5 # Threshold to detect object
nms_thres = .5 # 0-1 higer value means lower suppress 

In [138]:
# resize a bit 
imgHeight, imgWidth, _ = img.shape
scale= 60
imgH = int (imgHeight *scale /100)
imgW = int (imgWidth * scale /100)
img = cv2.resize(img, (imgW, imgH))

In [147]:
img = cv2.imread(r'testImages/tb2.jpg')

In [148]:
classIds, confs, bbox = net.detect(img,confThreshold=thres)
type(confs),type(bbox)

(tuple, tuple)

In [149]:
#Formating data
bbox = list(bbox)
confs = list(np.array(confs).reshape(1, -1)[0])
confs = list(map(float,confs ))

indics = cv2.dnn.NMSBoxes(bbox, confs, thres, nms_thres) # remove overlaps

for ind in indics:
    i = ind[0] # ind returns list we need index only
    box = bbox[i]

    if classNames[classIds[i][0]-1] == "cell phone":
        cv2.rectangle(img, box, rectColor, 1) #set the rectangle around object
        
        rectBox = box
        print(box)
#         print()
        #show accuracy level
        cv2.putText(img, str(round(confs[i]*100, 1)), (box[0]+10, box[1]+10), 
                   font, fontSize, fontColor, 1)
        #putting name of object 
        cv2.putText(img, classNames[classIds[i][0]-1], (box[0]+70, box[1]+30), 
                   font, fontSize, fontColor,1)


cv2.imshow("Object detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [141]:
print(rectBox)
(x, y) = (rectBox[0], rectBox[1])
(w, h) = (rectBox[2], rectBox[3])
print(x, y, w, h)

[ 59 207 106 158]
59 207 106 158


In [142]:
# need make it flexiable 
detectW =  30
startX= x+(w//2) # center X
print(startX)

112


In [143]:
cv2.rectangle(img, (startX- detectW, y),(startX +detectW, y+50), (220,1,0),2 )
# cv2.imwrite("LabRatimage.jpg", img)
cv2.circle(img, (startX, y), 13, (0,0,222))
cv2.imshow("Object detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## <center> Cam version

In [162]:
cam = cv2.VideoCapture(0)
while True:
    success, img = cam.read()
    if success:
        classIds, confs, bbox = net.detect(img,confThreshold=thres)
        
        #Formating data
        bbox = list(bbox)
        confs = list(np.array(confs).reshape(1, -1)[0])
        confs = list(map(float,confs ))

        indics = cv2.dnn.NMSBoxes(bbox, confs, .1, nms_thres) # remove overlaps
        
        for ind in indics:
            i = ind[0] # ind returns list we need index only
            box = bbox[i]
            
            if classNames[classIds[i][0]-1] == "cell phone":
                cv2.rectangle(img, box, rectColor, 1) #set the rectangle around object

                #show accuracy level
                cv2.putText(img, str(round(confs[i]*100, 1)), (box[0]+10, box[1]+10), 
                           font, fontSize, fontColor, 1)
                #putting name of object 
                cv2.putText(img, classNames[classIds[i][0]-1], (box[0]+70, box[1]+30), 
                           font, fontSize, fontColor,1)

    
        cv2.imshow("Object detection", img)
        if cv2.waitKey(1) == 27: 
            break  # esc to quit
    
cam.release()
cv2.destroyAllWindows()